In [1]:
""" Union of hydrobasin and GADM 36 level 1 using geopandas parallel processing.
-------------------------------------------------------------------------------

Step 1:
Create polygons (10x10 degree, 648).

Step 2:
Clip all geodataframes with polygons (intersect).

Step 3:
Peform union per polygon.

Step 4: 
Merge results into large geodataframe.

Step 5:
Dissolve on unique identifier.

Step 6:
Save output

Author: Rutger Hofste
Date: 20181128
Kernel: python35+
Docker: rutgerhofste/gisdocker:ubuntu16.04

"""


TESTING = 1
SCRIPT_NAME = "Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01"
OUTPUT_VERSION = 19

RDS_DATABASE_ENDPOINT = "aqueduct30v05.cgpnumwmfcqc.eu-central-1.rds.amazonaws.com"
RDS_DATABASE_NAME = "database01"
RDS_INPUT_TABLE_LEFT = "y2018m11d12_rh_gadm36_level1_to_rds_v01_v02"
RDS_INPUT_TABLE_RIGHT = "hybas06_v04"

ec2_output_path = "/volumes/data/{}/output_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)
ec2_output_path_df = "/volumes/data/{}/outputdf_V{:02.0f}".format(SCRIPT_NAME,OUTPUT_VERSION)

s3_output_path = "s3://wri-projects/Aqueduct30/processData/{}/output_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)
s3_output_path_df = "s3://wri-projects/Aqueduct30/processData/{}/outputdf_V{:02.0f}/".format(SCRIPT_NAME,OUTPUT_VERSION)


print("\nec2_output_path:", ec2_output_path,
      "\ns3_output_path: ", s3_output_path)


ec2_output_path: /volumes/data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19 
s3_output_path:  s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/


In [2]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2018M12D03 UTC 14:42


'3.5.4 |Anaconda, Inc.| (default, Nov 20 2017, 18:44:38) \n[GCC 7.2.0]'

In [3]:
import os
import sqlalchemy
import multiprocessing
import numpy as np
import pandas as pd
import geopandas as gpd
from google.cloud import bigquery
from shapely.geometry import MultiPolygon, Polygon

In [4]:
!rm -r {ec2_output_path}
!rm -r {ec2_output_path}
!mkdir -p {ec2_output_path}
!mkdir -p {ec2_output_path_df}

rm: cannot remove '/volumes/data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19': No such file or directory
rm: cannot remove '/volumes/data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19': No such file or directory


In [5]:
cpu_count = multiprocessing.cpu_count()
cpu_count = cpu_count -2 #Avoid freeze
print("Power to the maxxx:", cpu_count)


Power to the maxxx: 14


In [6]:
F = open("/.password","r")
password = F.read().splitlines()[0]
F.close()

engine = sqlalchemy.create_engine("postgresql://rutgerhofste:{}@{}:5432/{}".format(password,RDS_DATABASE_ENDPOINT,RDS_DATABASE_NAME))

In [7]:
cell_size = 10


In [8]:
def create_fishnet_gdf(cell_size):
    crs = {'init': 'epsg:4326'}
    lats = np.arange(-90,90,cell_size)
    lons = np.arange(-180,180,cell_size)
    geoms = []
    for lat in lats:
        for lon in lons:
            llcr = (lon,lat)
            lrcr = (lon+cell_size, lat)
            urcr = (lon+cell_size, lat+cell_size)
            ulcr = (lon, lat+ cell_size)
            geom = Polygon([llcr,lrcr,urcr,ulcr])
            geoms.append(geom)
    gs = gpd.GeoSeries(geoms)
    gdf_grid = gpd.GeoDataFrame(geometry=gs)
    gdf_grid.crs = crs
    return gdf_grid

def post_process_geometry(geometry):
    """ Post Process Shapely Geometries after Intersection
    
    Shapely does not always create the desired output geometry. When
    vertices overlap, the result can be a geometryCollection with
    (mutli)polygons and LineStrings or Points. 
    
    This function converts the results of an intersection. It will
    remove empty geometries
    
    Args: 
        SIMPLIFY_TOLERANCE(double): Global parameter to specify 
            simplification tolerance.
        geomerty (shapely object): GeometryCollection, Multipolygon
            Polygon, Linestring etc.
            
    Returns:
        geometry_out(shapely object): MultiPolygon or
            Polygon of simplified geometry.
            
    Usage:
        apply to geodataframe geometry column.
    
    """
    geometry_buffered = geometry.buffer(0)
    return geometry_buffered


def clip_gdf(gdf_in,polygon):
    """
    Clip geodataframe using shapely polygon.
    Make sure crs is compatible.
    
    Removes any geometry that is (multi)polygon. i.e. LineStrings and Points are Removed
    
    Args:
        gdf (GeoDataFrame): GeoDataFrame in.
        polygon (Shapely Polygon): Polygon used for clipping
    
    """
    crs = gdf_in.crs
    gdf_intersects = gdf_in.loc[gdf_in.geometry.intersects(polygon)]
    df_intersects = gdf_intersects.drop(columns=[gdf_intersects.geometry.name])
    gs_intersections = gpd.GeoSeries(gdf_intersects.geometry.intersection(polygon),crs=crs)
    gdf_clipped = gpd.GeoDataFrame(df_intersects,geometry=gs_intersections)  
    
    # Some clipping results in GeometryCollections with polygons and LineStrings or Points. Convert valid geometry to Multipolygon
    gdf_clipped.geometry = gdf_clipped.geometry.apply(post_process_geometry)
    gdf_clipped_valid = gdf_clipped.loc[gdf_clipped.geometry.is_empty == False]
    return gdf_clipped_valid


def create_union_gdfs(gdf):
    index = gdf.index[0]
    print("Processing: ", index)
    df_out = pd.DataFrame()
    start = datetime.datetime.now()
    polygon = gdf.iloc[0].geometry
    destination_path = "{}/gdf_union_{}.pkl".format(ec2_output_path,index)
    
    gdf_left_clipped= clip_gdf(gdf_left,polygon)
    gdf_right_clipped = clip_gdf(gdf_right,polygon)
    
    if gdf_left_clipped.shape[0] == 0 and gdf_right_clipped.shape[0] == 0:

        gdf_out = None
        write_output = False
    elif gdf_left_clipped.shape[0] != 0 and gdf_right_clipped.shape[0] == 0:
        gdf_out = gdf_left_clipped
        write_output = True
    elif gdf_left_clipped.shape[0] == 0 and gdf_right_clipped.shape[0] != 0:
        gdf_out = gdf_right_clipped
        write_output = True
    elif gdf_left_clipped.shape[0] != 0 and gdf_right_clipped.shape[0] != 0:        
        gdf_union = gpd.overlay(gdf_left_clipped,gdf_right_clipped,how="union")
        gdf_out = gdf_union
        write_output = True        
    
    
    if write_output:
        if TESTING:
            end = datetime.datetime.now()
            elapsed = end - start
            gdf_out["time_processed"] = elapsed.total_seconds()
            gdf_out["tile_index"] = index
        else:
            pass
        gdf_out.to_pickle(path=destination_path)
    else:
        pass
    
    print("Succesfully processed", index)
    return  gdf_out
        

In [9]:
gdf_grid = create_fishnet_gdf(cell_size)

In [10]:
gdf_grid.head()

,geometry
0,"POLYGON ((-180 -90, -170 -90, -170 -80, -180 -..."
1,"POLYGON ((-170 -90, -160 -90, -160 -80, -170 -..."
2,"POLYGON ((-160 -90, -150 -90, -150 -80, -160 -..."
3,"POLYGON ((-150 -90, -140 -90, -140 -80, -150 -..."
4,"POLYGON ((-140 -90, -130 -90, -130 -80, -140 -..."


In [11]:
gdf_grid.shape

(648, 1)

In [12]:
sql = """
SELECT
  gid_1,
  geom
FROM
  {}
""".format(RDS_INPUT_TABLE_LEFT)

In [13]:
gdf_left = gpd.read_postgis(sql=sql,
                            con=engine)

In [14]:
gdf_left.head()

,gid_1,geom
0,AFG.10_1,"(POLYGON ((64.52828217 33.32641602, 64.5058975..."
1,AFG.1_1,"(POLYGON ((71.14804076999999 36.00123596, 71.1..."
2,AFG.11_1,"(POLYGON ((63.6648941 29.4764061, 63.66107941 ..."
3,AFG.12_1,"(POLYGON ((62.26362991 32.84782028, 62.2033577..."
4,AFG.15_1,"(POLYGON ((66.19773102000001 31.86677933, 66.1..."


In [15]:
gdf_left.shape

(3610, 2)

In [16]:
sql = """
SELECT
  pfaf_id,
  geom
FROM
  {}
""".format(RDS_INPUT_TABLE_RIGHT)

In [17]:
gdf_right = gpd.read_postgis(sql=sql,
                             con=engine)

In [18]:
gdf_right.head()

,pfaf_id,geom
0,111015,(POLYGON ((34.00000000000003 26.62500000000002...
1,111019,(POLYGON ((36.24444444444446 23.55833333333337...
2,111020,(POLYGON ((35.14166666666669 22.58333333333336...
3,111041,(POLYGON ((36.52916666666668 21.25000000000003...
4,111050,(POLYGON ((37.19583333333335 20.00416666666668...


In [19]:
gdf_right.shape

(16396, 2)

In [20]:
# inspect case for Egypt index = 417, case for Canada index = 515
gdf_grid.head()

,geometry
0,"POLYGON ((-180 -90, -170 -90, -170 -80, -180 -..."
1,"POLYGON ((-170 -90, -160 -90, -160 -80, -170 -..."
2,"POLYGON ((-160 -90, -150 -90, -150 -80, -160 -..."
3,"POLYGON ((-150 -90, -140 -90, -140 -80, -150 -..."
4,"POLYGON ((-140 -90, -130 -90, -130 -80, -140 -..."


In [21]:
#gdf_grid_list = np.array_split(gdf_grid, cpu_count*10)
gdf_grid_list = np.array_split(gdf_grid, gdf_grid.shape[0])

In [22]:
len(gdf_grid_list)

648

In [23]:
p= multiprocessing.Pool(processes=cpu_count)
df_out_list = p.map(create_union_gdfs,gdf_grid_list)
p.close()
p.join()

Processing:  0
Processing:  24
Processing:  12
Processing:  36
Processing:  60
Processing:  48
Processing:  84
Processing:  72
Processing:  144
Processing:  156
Processing:  108
Processing:  120
Processing:  96
Processing:  132
Succesfully processed 60
Processing:  61
Succesfully processed 156
Processing:  157
Succesfully processed 84
Processing:  85
Succesfully processed 0
Processing:  1
Succesfully processed 12
Processing:  13
Succesfully processed 108
Processing:  109
Succesfully processed 36
Processing:  37
Succesfully processed 24
Processing:  25
Succesfully processed 48
Processing:  49
Succesfully processed 72
Processing:  73
Succesfully processed 144
Processing:  145
Succesfully processed 96
Processing:  97
Succesfully processed 61
Processing:  62
Succesfully processed 132
Processing:  133
Succesfully processed 120
Succesfully processed 157
Processing:  121
Processing:  158
Succesfully processed 85
Processing:  86
Succesfully processed 13
Processing:  14
Succesfully processed 14

Succesfully processed 208
Processing:  209
Processing:  220
Succesfully processed 291
Processing:  292
Succesfully processed 171
Processing:  172
Succesfully processed 278
Processing:  279
Succesfully processed 243
Processing:  244
Succesfully processed 220
Processing:  221
Succesfully processed 185
Processing:  186
Succesfully processed 292
Processing:  293
Succesfully processed 255
Processing:  256
Succesfully processed 172
Processing:  173
Succesfully processed 279
Processing:  280
Succesfully processed 244
Processing:  245
Succesfully processed 186
Processing:  187
Succesfully processed 221
Processing:  222
Succesfully processed 293
Processing:  294
Succesfully processed 256
Processing:  257
Succesfully processed 173
Processing:  174
Succesfully processed 280
Processing:  281
Succesfully processed 222
Processing:  223
Succesfully processed 187
Processing:  188
Succesfully processed 294
Processing:  295
Succesfully processed 174
Processing:  175
Succesfully processed 295
Processing:

Processing:  437
Succesfully processed 445
Processing:  446
Succesfully processed 446
Processing:  447
Succesfully processed 447
Processing:  448
Succesfully processed 448
Processing:  449
Succesfully processed 437
Processing:  438
Succesfully processed 438
Processing:  439
Succesfully processed 439
Processing:  440
Succesfully processed 178
Processing:  179
Succesfully processed 440
Processing:  441
Succesfully processed 417
Processing:  418
Succesfully processed 371
Processing:  456
Succesfully processed 456
Processing:  457
Succesfully processed 441
Processing:  442
Succesfully processed 404
Processing:  405
Succesfully processed 457
Processing:  458
Succesfully processed 298
Processing:  299
Succesfully processed 418
Processing:  419
Succesfully processed 353
Processing:  354
Succesfully processed 420
Processing:  421
Succesfully processed 299
Processing:  468
Succesfully processed 468
Processing:  469
Succesfully processed 469
Processing:  470
Succesfully processed 470
Processing:

Processing:  548
Succesfully processed 646
Processing:  647
Succesfully processed 647
Succesfully processed 604
Processing:  605
Succesfully processed 605
Processing:  606
Succesfully processed 508
Processing:  509
Succesfully processed 487
Processing:  488
Succesfully processed 463
Processing:  464
Succesfully processed 606
Processing:  607
Succesfully processed 553
Processing:  554
Succesfully processed 607
Processing:  608
Succesfully processed 608
Processing:  609
Succesfully processed 609
Processing:  610
Succesfully processed 610
Processing:  611
Succesfully processed 611
Succesfully processed 464
Processing:  465
Succesfully processed 465
Processing:  466
Succesfully processed 466
Processing:  467
Succesfully processed 467
Succesfully processed 554
Processing:  555
Succesfully processed 555
Processing:  556
Succesfully processed 591
Processing:  592
Succesfully processed 592
Processing:  593
Succesfully processed 556
Processing:  557
Succesfully processed 593
Processing:  594
Su

In [24]:
df_out = pd.concat(df_out_list, ignore_index=True)

In [25]:
output_path_df = "{}/df_out.pkl".format(ec2_output_path_df)

In [26]:
df_out.to_pickle(output_path_df)

In [27]:
!aws s3 cp {ec2_output_path} {s3_output_path} --recursive

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_132.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_132.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_160.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_160.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_141.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_141.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_120.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_120.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_161.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_227.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_227.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_228.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_228.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_212.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_212.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_231.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_231.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_229.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_285.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_285.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_289.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_289.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_288.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_288.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_287.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_287.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_282.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_340.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_340.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_344.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_344.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_334.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_334.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_347.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_347.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_343.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_389.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_389.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_382.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_382.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_391.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_391.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_387.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_387.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_394.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_443.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_443.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_440.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_440.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_446.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_446.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_441.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_441.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_447.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_495.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_495.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_494.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_494.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_497.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_497.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_496.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_496.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_487.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_523.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_523.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_538.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_538.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_539.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_539.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_542.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_542.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_536.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_577.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_577.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_576.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_576.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_578.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_578.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_579.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_579.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_575.pkl to s3://wri-projects/Aqueduct30/p

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_624.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_624.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_626.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_626.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_625.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_625.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_627.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_627.pkl
upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/output_V19/gdf_union_629.pkl to s3://wri-projects/Aqueduct30/p

In [28]:
!aws s3 cp {ec2_output_path_df} {s3_output_path_df} --recursive

upload: ../../../../data/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/outputdf_V19/df_out.pkl to s3://wri-projects/Aqueduct30/processData/Y2018M11D29_RH_Hybas6_U_GADM36L01_GPD_PP_V01/outputdf_V19/df_out.pkl


In [29]:
end = datetime.datetime.now()
elapsed = end - start
print(elapsed)

0:46:44.060076


Previous Runs:  
0:45:12.187817 (10x10)  
0:44:55.686081 (10x10)  
1:16:26.394030 (1x1 sindex=true)  
0:47:42.023796  
